In [13]:
import json
import os

In [ ]:
json_path = 'instances_images.json'
output_dir = 'annotations'

In [21]:
def coco_to_yolo_bbox(bbox, img_w, img_h):
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w /= img_w
    h /= img_h
    return [x_center, y_center, w, h]

In [26]:
def convert_coco_json_to_yolo(json_path, output_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)
        
    os.makedirs(output_dir, exist_ok=True)
    
    categories = {cat['id']: i for i, cat in enumerate(data["categories"])}
    
    images = {img['id']: img for img in data["images"]}
    
    ann_by_image = {}
    for ann in data["annotations"]:
        img_id = ann["image_id"]
        if img_id not in ann_by_image:
            ann_by_image[img_id] = []
        ann_by_image[img_id].append(ann)
        
    for img_id, anns in ann_by_image.items():
        img = images[img_id]
        img_w, img_h = img["width"], img["height"]
        filename = os.path.splitext(img["file_name"])[0] + ".txt"
        label_path = os.path.join(output_dir, filename)
        
        with open(label_path, 'w') as f:
            for ann in anns:
                class_id = categories[ann['category_id']]
                yolo_bbox = coco_to_yolo_bbox(ann["bbox"], img_w, img_h)
                bbox_str = ' '.join([str(round(coord, 6)) for coord in yolo_bbox])
                f.write(f"{class_id} {bbox_str}\n")
    
    print(f"Conversion Complete! YOLO labels saved to {output_dir}") 

In [27]:
convert_coco_json_to_yolo(json_path, output_dir)

Conversion Complete! YOLO labels saved to annotations


In [ ]:
import shutil
img_dir = 'images'
label_dir = 'annotations'
images = [os.path.splitext(img)[0] for img in os.listdir(img_dir)]
labels = [os.path.splitext(label)[0] for label in os.listdir(label_dir)]
diff = [file + '.jpg' for file in images if file not in labels]
dir = 'unannotated images'
os.makedirs(dir, exist_ok=True)
for file in diff:
    src = os.path.join(img_dir, file)
    dest = os.path.join(dir, file)
    if os.path.exists(src):
        shutil.move(src, dest)
        print(f"Moved: {file}")
    else:
        print(f"File not found: {file}")

Moved: 000000025593.jpg
Moved: 000000041488.jpg
Moved: 000000042888.jpg
Moved: 000000049091.jpg
Moved: 000000058636.jpg
Moved: 000000064574.jpg
Moved: 000000098497.jpg
Moved: 000000101022.jpg
Moved: 000000121153.jpg
Moved: 000000127135.jpg
Moved: 000000173183.jpg
Moved: 000000176701.jpg
Moved: 000000198915.jpg
Moved: 000000200152.jpg
Moved: 000000226111.jpg
Moved: 000000228771.jpg
Moved: 000000240767.jpg
Moved: 000000260657.jpg
Moved: 000000261796.jpg
Moved: 000000267946.jpg
Moved: 000000268996.jpg
Moved: 000000270386.jpg
Moved: 000000278006.jpg
Moved: 000000308391.jpg
Moved: 000000310622.jpg
Moved: 000000312549.jpg
Moved: 000000320706.jpg
Moved: 000000330554.jpg
Moved: 000000344611.jpg
Moved: 000000370999.jpg
Moved: 000000374727.jpg
Moved: 000000382734.jpg
Moved: 000000402096.jpg
Moved: 000000404601.jpg
Moved: 000000447789.jpg
Moved: 000000458790.jpg
Moved: 000000461275.jpg
Moved: 000000476491.jpg
Moved: 000000477118.jpg
Moved: 000000481404.jpg
Moved: 000000502910.jpg
Moved: 000000514

In [3]:
import json
with open('json annotations/instances_images.json', 'r') as f:
    labels = json.load(f)
    
categories = {cat['name']: i for i, cat in enumerate(labels["categories"])}
len(categories)

80

In [18]:
import shutil
anns = [label for label in os.listdir("unannotated images") if label.endswith('.json')]
imgs = [os.path.splitext(file)[0] + ".jpg" for file in anns]


for x in imgs:
    srcx = os.path.join("unannotated images", x)
    destx = os.path.join("images", x)
    shutil.move(srcx, destx)
    print(f"Moved: {x}")

Moved: 000000025593.jpg
Moved: 000000041488.jpg
Moved: 000000042888.jpg
Moved: 000000049091.jpg
Moved: 000000058636.jpg
Moved: 000000064574.jpg
Moved: 000000098497.jpg
Moved: 000000101022.jpg
Moved: 000000121153.jpg
Moved: 000000127135.jpg
Moved: 000000173183.jpg
Moved: 000000176701.jpg
Moved: 000000198915.jpg
Moved: 000000200152.jpg
Moved: 000000226111.jpg
Moved: 000000228771.jpg
Moved: 000000260657.jpg
Moved: 000000261796.jpg
Moved: 000000267946.jpg
Moved: 000000268996.jpg
Moved: 000000270386.jpg
Moved: 000000278006.jpg
Moved: 000000308391.jpg
Moved: 000000310622.jpg
Moved: 000000312549.jpg
Moved: 000000344611.jpg
Moved: 000000370999.jpg
Moved: 000000374727.jpg
Moved: 000000402096.jpg
Moved: 000000404601.jpg
Moved: 000000447789.jpg
Moved: 000000461275.jpg
Moved: 000000476491.jpg
Moved: 000000477118.jpg
Moved: 000000502910.jpg
Moved: 000000514540.jpg
Moved: 000000528977.jpg
Moved: 000000542073.jpg
Moved: 000000550939.jpg
Moved: 000000556498.jpg
Moved: 000000560371.jpg


In [15]:
import os
import json
def convert_labelme_to_yolo(json_file, output_dir="annotations"):
    def get_yolo_bbox(points, img_w, img_h):
        (x1, y1), (x2, y2) = points
        x_min , x_max = min(x1, x2), max(x1, x2)
        y_min , y_max = min(y1, y2), max(y1, y2)
        
        x_center = (x_max + x_min) / (2 * img_w)
        y_center = (y_max + y_min) / (2 * img_h)
        w = (x_max - x_min) / img_w
        h = (y_max - y_min) / img_h
        
        return [x_center, y_center, w, h]
    
    def get_classid(label):
        if label not in categories:
            categories[label] = len(categories)
        
        return categories[label]
    
    with open(os.path.join("unannotated images", json_file), 'r') as f:
        annotation = json.load(f)
        
    img_w, img_h = annotation["imageWidth"], annotation["imageHeight"]
    
    yolo_labels = []
    
    for shape in annotation["shapes"]:
        class_id = get_classid(shape["label"])
        bbox = get_yolo_bbox(shape["points"], img_w, img_h)
        yolo_labels.append(f"{class_id} {' '.join([str(round(coord, 6)) for coord in bbox])}")
    
    filename = os.path.splitext(json_file)[0] + ".txt"
    label_path = os.path.join(output_dir, filename)
    with open(label_path, 'w') as f:
        f.write('\n'.join(yolo_labels))   


In [16]:
output_dir="unannotated images"
for file in os.listdir("unannotated images"):
    convert_labelme_to_yolo(file, output_dir)
    
print(f"Conversion complete! YOLO labels saved to {output_dir}")

Conversion complete! YOLO labels saved to unannotated images


In [18]:
import shutil
for file in os.listdir(output_dir):
    if file.endswith(".txt"):
        src = os.path.join(output_dir, file)
        dest = os.path.join("annotations", file)
        shutil.move(src, dest)

In [1]:
def get_yolo_bbox(bbox, img_w, img_h):
        x1, y1, x2, y2 = bbox
        x_min , x_max = min(x1, x2), max(x1, x2)
        y_min , y_max = min(y1, y2), max(y1, y2)
        
        x_center = (x_max + x_min) / (2 * img_w)
        y_center = (y_max + y_min) / (2 * img_h)
        w = (x_max - x_min) / img_w
        h = (y_max - y_min) / img_h
        
        return [x_center, y_center, w, h]

In [3]:
import json
import os
import cv2

path = 'dataset/annotations'

for file in os.listdir(path):
    print(f"Processing: {file}")
    
    label_path = os.path.join(path, file)
    img_file = os.path.splitext(file)[0] + '.png'
    img_path = os.path.join('dataset/images', img_file)
    
    img = cv2.imread(img_path)
    img_h, img_w, _ = img.shape
    
    with open(label_path, 'r', encoding='utf-8') as f:
        label = json.load(f)
    
    yolo_labels = []
    
    for ann in label["form"]:
        bbox = get_yolo_bbox(ann["box"], img_w, img_h)
            
        yolo_labels.append(f"80 {' '.join([str(round(coord, 6)) for coord in bbox])}")
           
    ann_path = os.path.splitext(file)[0] + ".txt"
    ann_path = os.path.join('dataset', ann_path)
    with open(ann_path, 'w') as f:
        f.write('\n'.join(yolo_labels))
            

Processing: 0000971160.json
Processing: 0000989556.json
Processing: 0000990274.json
Processing: 0000999294.json
Processing: 0001118259.json
Processing: 0001123541.json
Processing: 0001129658.json
Processing: 0001209043.json
Processing: 0001239897.json
Processing: 0001438955.json
Processing: 0001456787.json
Processing: 0001463282.json
Processing: 0001463448.json
Processing: 0001476912.json
Processing: 0001477983.json
Processing: 0001485288.json
Processing: 00040534.json
Processing: 00070353.json
Processing: 00093726.json
Processing: 0011505151.json
Processing: 0011838621.json
Processing: 0011845203.json
Processing: 0011856542.json
Processing: 0011859695.json
Processing: 0011899960.json
Processing: 0011906503.json
Processing: 0011973451.json
Processing: 0011974919.json
Processing: 0011976929.json
Processing: 0012178355.json
Processing: 0012199830.json
Processing: 0012529284.json
Processing: 0012529295.json
Processing: 0012602424.json
Processing: 0012947358.json
Processing: 0013255595.jso

In [ ]:
import os
def get_classid(label):
        if label not in categories:
            categories[label] = len(categories)
           
def helper(file):
    path = os.path.join(input_dir, file)
    with open(path, 'r') as f:
        annotation = json.load(f)
    
    for shape in annotation["shapes"]:
        get_classid(shape["label"])
    
input_dir = "json annotations"
for file in os.listdir(input_dir)[:-1]:
    helper(file)
    
len(categories)
categories
    

Labels extracted from 000000025593.json
Labels extracted from 000000041488.json
Labels extracted from 000000042888.json
Labels extracted from 000000049091.json
Labels extracted from 000000058636.json
Labels extracted from 000000064574.json
Labels extracted from 000000098497.json
Labels extracted from 000000101022.json
Labels extracted from 000000121153.json
Labels extracted from 000000127135.json
Labels extracted from 000000173183.json
Labels extracted from 000000176701.json
Labels extracted from 000000198915.json
Labels extracted from 000000200152.json
Labels extracted from 000000226111.json
Labels extracted from 000000228771.json
Labels extracted from 000000260657.json
Labels extracted from 000000261796.json
Labels extracted from 000000267946.json
Labels extracted from 000000268996.json
Labels extracted from 000000270386.json
Labels extracted from 000000278006.json
Labels extracted from 000000308391.json
Labels extracted from 000000310622.json
Labels extracted from 000000312549.json


{'person': 0,
 'bicycle': 1,
 'car': 2,
 'motorcycle': 3,
 'airplane': 4,
 'bus': 5,
 'train': 6,
 'truck': 7,
 'boat': 8,
 'traffic light': 9,
 'fire hydrant': 10,
 'stop sign': 11,
 'parking meter': 12,
 'bench': 13,
 'bird': 14,
 'cat': 15,
 'dog': 16,
 'horse': 17,
 'sheep': 18,
 'cow': 19,
 'elephant': 20,
 'bear': 21,
 'zebra': 22,
 'giraffe': 23,
 'backpack': 24,
 'umbrella': 25,
 'handbag': 26,
 'tie': 27,
 'suitcase': 28,
 'frisbee': 29,
 'skis': 30,
 'snowboard': 31,
 'sports ball': 32,
 'kite': 33,
 'baseball bat': 34,
 'baseball glove': 35,
 'skateboard': 36,
 'surfboard': 37,
 'tennis racket': 38,
 'bottle': 39,
 'wine glass': 40,
 'cup': 41,
 'fork': 42,
 'knife': 43,
 'spoon': 44,
 'bowl': 45,
 'banana': 46,
 'apple': 47,
 'sandwich': 48,
 'orange': 49,
 'broccoli': 50,
 'carrot': 51,
 'hot dog': 52,
 'pizza': 53,
 'donut': 54,
 'cake': 55,
 'chair': 56,
 'couch': 57,
 'potted plant': 58,
 'bed': 59,
 'dining table': 60,
 'toilet': 61,
 'tv': 62,
 'laptop': 63,
 'mouse':